## Run MOFAcell

In [ ]:
from mudata import MuData
import muon as mu
import mofax as mofa

In [ ]:
adata = sc.concat(adatas=adatas)
adata.obs['leiden'] = adata.obs['leiden'].replace({'5':'2'})

In [ ]:
mdata = li.multi.adata_to_views(adata,
                                groupby='leiden',
                                sample_key='sample',
                                obs_keys=None,
                                min_prop=0.05, # min nnz values (filter features]
                                min_smpls=3, # min samples per view (filter features]
                                min_cells=25, # min cells per view (filter samples]
                                min_counts=100, # min counts per view (filter samples]
                                mode='sum', # mode of aggregation
                                verbose=True,
                                skip_checks=True,
                                layer='counts'
                                )

In [ ]:
for mod in mdata.mod.keys():
    sc.pp.normalize_total(mdata.mod[mod], target_sum=1e4)
    sc.pp.log1p(mdata.mod[mod])
    
    sc.pp.highly_variable_genes(mdata.mod[mod], min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
model = mu.tl.mofa(mdata, 
                   use_obs='union',
                   convergence_mode='medium',
                   outfile="models/mofacell.hdf5",
                   n_factors=4,
                   use_var='highly_variable'
                   )

In [ ]:
# obtain factor scores
factor_scores = li.multi.get_factor_scores(mdata, obsm_key='X_mofa')
factor_scores.head(]

In [ ]:
model = mofa.mofa_model("models/mofacell.hdf5")
model

In [ ]:
mofa.plot_r2(model, x='View')

In [ ]:
model.close()

## Run MOFAtalk

In [ ]:
lrdata.obs['group'] = lrdata.obs['sample'].str.split('_').str[0]

In [ ]:
mdata = li.multi.adata_to_views(lrdata,
                                groupby='leiden',
                                sample_key='sample',
                                obs_keys=['group'],
                                min_prop=0.05, # min nnz values (filter features]
                                min_smpls=3, # min samples per view (filter features]
                                min_cells=25, # min cells per view (filter samples]
                                min_counts=0, # min counts per view (filter samples]
                                mode='mean', # mode of aggregation
                                verbose=True,
                                skip_checks=True
                                )

In [ ]:
mdata

In [ ]:
model = mu.tl.mofa(mdata, 
                   use_obs='union',
                   convergence_mode='medium',
                   outfile="models/lrs.h5ad",
                   n_factors=4
                   )

In [ ]:
# obtain factor scores
factor_scores = li.multi.get_factor_scores(mdata, obsm_key='X_mofa')
factor_scores.head()


In [ ]:
model = mofa.mofa_model("models/lrs.h5ad")
model

In [ ]:
mofa.plot_r2(model, x='View')

In [ ]:
factor1_loadings =  li.multi.get_variable_loadings(mdata, 0, view_separator=':') # get loadings for factor 1
## NOTE this should work, not only with 0...

In [ ]:
factor1_loadings.head(10)

Test Tensor GPU